# A Conceptual, Practical Introduction to Trax Layers

This notebook introduces the core concepts of the Trax library through a series of code samples and explanations. The topics covered in following sections are:

  1. **Layers**: the basic building blocks and how to combine them into networks
  1. **Data Streams**: how individual layers manage inputs and outputs
  1. **Data Stack**: how the Trax runtime manages data streams for the layers
  1. **Defining New Layer Classes**: how to define and test your own layer classes
  1. **Models**: how to train, evaluate, and run predictions with Trax models




## General Setup
Execute the following few cells (once) before running any of the code samples in this notebook.

In [0]:
# Copyright 2018 Google LLC.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import numpy as np




In [0]:
# Import Trax

! pip install -q -U trax
! pip install -q tensorflow

from trax import math
from trax import layers as tl
from trax import shapes
from trax.math import numpy as jnp  # For use in defining new layer types.
from trax.shapes import ShapeDtype
from trax.shapes import signature

/bin/sh: pip: command not found
/bin/sh: pip: command not found


In [0]:
# Settings and utilities for handling inputs, outputs, and object properties.

np.set_printoptions(precision=3)  # Reduce visual noise from extra digits.

def show_layer_properties(layer_obj, layer_name):
  template = ('{}.n_in:  {}\n'
              '{}.n_out: {}\n'
              '{}.sublayers: {}\n'
              '{}.weights:    {}\n')
  print(template.format(layer_name, layer_obj.n_in,
                        layer_name, layer_obj.n_out,
                        layer_name, layer_obj.sublayers,
                        layer_name, layer_obj.weights))

def floats_in_range(start, end):
  return np.arange(start, end).astype(np.float32)

# 1. Layers

The Layer class represents Trax's basic building blocks:
```
class Layer(object):
  """Base class for composable layers in a deep learning network.

  Layers are the basic building blocks for deep learning models. A Trax layer
  computes a function from zero or more inputs to zero or more outputs,
  optionally using trainable weights (common) and non-parameter state (not
  common). Authors of new layer subclasses typically override at most two
  methods of the base `Layer` class:

    forward(inputs, weights):
      Computes this layer's output as part of a forward pass through the model.

    new_weights(self, input_signature):
      Returns new weights suitable for inputs with the given signature.

  ...
```

## A layer computes a function.

A layer computes a function from zero or more inputs to zero or more outputs.
The inputs and outputs are NumPy arrays or JAX objects behaving as NumPy arrays.

The simplest layers, those with no weights or sublayers, can be used without
initialization. You can think of them (and test them) like pure mathematical
functions. For ease of testing and interactive exploration, layer objects
implement the `__call__ ` method, so you can call them directly on input data:
```
y = my_layer(x)
```

Layers are also objects, so you can inspect their properties. For example:
```
print('Number of inputs expected by this layer: {}'.format(my_layer.n_in))
```

### Example 1. tl.Relu $[n_{in} = 1, n_{out} = 1]$

In [0]:
relu = tl.Relu()

x = floats_in_range(-7, 8).reshape(3, -1)
y = relu(x)

# Show input, output, and two layer properties.
print(f'x:\n{x}\n\n'
      f'relu(x):\n{y}\n\n'
      f'number of inputs expected by this layer: {relu.n_in}\n'
      f'number of outputs promised by this layer: {relu.n_out}')

x:
[[-7. -6. -5. -4. -3.]
 [-2. -1.  0.  1.  2.]
 [ 3.  4.  5.  6.  7.]]

relu(x):
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 2.]
 [3. 4. 5. 6. 7.]]

number of inputs expected by this layer: 1
number of outputs promised by this layer: 1


### Example 2. tl.Concatenate $[n_{in} = 2, n_{out} = 1]$

In [0]:
concat_axis_0 = tl.Concatenate(axis=0)
concat_axis_1 = tl.Concatenate(axis=1)

x1 = floats_in_range(-7, 8).reshape(3, -1)
x2 = x1 * 10.
y0 = concat_axis_0([x1, x2])
y1 = concat_axis_1([x1, x2])

print(f'x1:\n{x1}\n\n'
      f'x2:\n{x2}\n\n'
      f'concat_axis_0([x1, x2]):\n{y0}\n\n'
      f'concat_axis_1([x1, x2]):\n{y1}\n')

# Print abbreviated object representations (useful for debugging).
print(f'concat_axis_0: {concat_axis_0}')
print(f'concat_axis_1: {concat_axis_1}')

x1:
[[-7. -6. -5. -4. -3.]
 [-2. -1.  0.  1.  2.]
 [ 3.  4.  5.  6.  7.]]

x2:
[[-70. -60. -50. -40. -30.]
 [-20. -10.   0.  10.  20.]
 [ 30.  40.  50.  60.  70.]]

concat_axis_0([x1, x2]):
[[ -7.  -6.  -5.  -4.  -3.]
 [ -2.  -1.   0.   1.   2.]
 [  3.   4.   5.   6.   7.]
 [-70. -60. -50. -40. -30.]
 [-20. -10.   0.  10.  20.]
 [ 30.  40.  50.  60.  70.]]

concat_axis_1([x1, x2]):
[[ -7.  -6.  -5.  -4.  -3. -70. -60. -50. -40. -30.]
 [ -2.  -1.   0.   1.   2. -20. -10.   0.  10.  20.]
 [  3.   4.   5.   6.   7.  30.  40.  50.  60.  70.]]

concat_axis_0: Concatenate_in2
concat_axis_1: Concatenate_in2


## Layers are trainable.

Most layer types include weights that affect the computation of outputs from inputs, and they use back-progagated gradients to update those weights.

🚧🚧 *A very small subset of layer types, such as `BatchNorm`, also include weights (called `state`) that are updated based on forward-pass inputs/computation rather than back-propagated gradients.*

### Initialization

Trainable layers must be initialized before use. Trax's model trainers take care of this as part of the overall training process. In other settings (e.g., in tests or interactively in a Colab notebook), you need to initialize the *outermost/topmost* layer explicitly. For this, use `init`:

```
  def init(self, input_signature, rng=None):
    """Initializes this layer and its sublayers recursively.

    This method is designed to initialize each layer instance once, even if the
    same layer instance occurs in multiple places in the network. This enables
    weight sharing to be implemented as layer sharing.

    Args:
      input_signature: A `ShapeDtype` instance (if this layer takes one input)
          or a list/tuple of `ShapeDtype` instances.
      rng: A single-use random number generator (JAX PRNG key). If none is
          provided, a default rng based on the integer seed 0 will be used.

    Returns:
      A (weights, state) tuple, in which weights contains newly created weights
          on the first call and `EMPTY_WEIGHTS` on all subsequent calls.
    """
```

Input signatures can be built from scratch using `ShapeDType` objects, or can
be derived from data via the `signature` function (in module `shapes`):
```
def signature(obj):
  """Returns a `ShapeDtype` signature for the given `obj`.

  A signature is either a `ShapeDtype` instance or a tuple of `ShapeDtype`
  instances. Note that this function is permissive with respect to its inputs
  (accepts lists or tuples, and underlying objects can be any type as long as
  they have shape and dtype attributes), but strict with respect to its outputs
  (only `ShapeDtype`, and only tuples).

  Args:
    obj: An object that has `shape` and `dtype` attributes, or a list/tuple
        of such objects.
  """
```

### Example 3. tl.LayerNorm $[n_{in} = 1, n_{out} = 1]$

In [0]:
layer_norm = tl.LayerNorm()

x = floats_in_range(-7, 8).reshape(3, -1)
layer_norm.init(signature(x))

y = layer_norm(x)

print(f'x:\n{x}\n\n'
      f'layer_norm(x):\n{y}\n')
print(f'layer_norm.weights:\n{layer_norm.weights}')

x:
[[-7. -6. -5. -4. -3.]
 [-2. -1.  0.  1.  2.]
 [ 3.  4.  5.  6.  7.]]

layer_norm(x):
[[-1.414 -0.707  0.     0.707  1.414]
 [-1.414 -0.707  0.     0.707  1.414]
 [-1.414 -0.707  0.     0.707  1.414]]

layer_norm.weights:
(DeviceArray([1., 1., 1., 1., 1.], dtype=float32), DeviceArray([0., 0., 0., 0., 0.], dtype=float32))


## Layers combine into layers.

The Trax library authors encourage users to build new layers as combinations of existing layers. Hence, the library provides a small set of _combinator_ layers: layer objects that make a list of layers behave as a single layer.

The new layer, like other layers, can:
* compute outputs from inputs,
* update parameters from gradients, and
* combine with yet more layers.

### Combine with `Serial`

The most common way to combine layers is with the `Serial` class:
```
class Serial(base.Layer):
  """Combinator that applies layers serially (by function composition).

  A Serial combinator uses stack semantics to manage data for its sublayers.
  Each sublayer sees only the inputs it needs and returns only the outputs it
  has generated. The sublayers interact via the data stack. For instance, a
  sublayer k, following sublayer j, gets called with the data stack in the
  state left after layer j has applied. The Serial combinator then:

    - takes n_in items off the top of the stack (n_in = k.n_in) and calls
      layer k, passing those items as arguments; and

    - takes layer k's n_out return values (n_out = k.n_out) and pushes
      them onto the data stack.

  ...
```
If one layer has the same number of outputs as the next layer has inputs (which is the usual case), the successive layers behave like function composition:

```
#  h(.) = g(f(.))
layer_h = Serial(
    layer_f,
    layer_g,
)
```
Note how, inside `Serial`, function composition is expressed naturally as a succession of operations, so that no nested parentheses are needed.


### Example 4. y = layer_norm(relu(x)) $[n_{in} = 1, n_{out} = 1]$

In [0]:
layer_block = tl.Serial(
    tl.Relu(),
    tl.LayerNorm(),
)

x = floats_in_range(-7, 8).reshape(3, -1)
layer_block.init(signature(x))
y = layer_block(x)

print(f'x:\n{x}\n\n'
      f'layer_block(x):\n{y}')

x:
[[-7. -6. -5. -4. -3.]
 [-2. -1.  0.  1.  2.]
 [ 3.  4.  5.  6.  7.]]

layer_block(x):
[[ 0.     0.     0.     0.     0.   ]
 [-0.75  -0.75  -0.75   0.5    1.75 ]
 [-1.414 -0.707  0.     0.707  1.414]]


And we can inspect the block as a whole, as if it were just another layer:

### Example 4'. Inspecting a Serial layer.

In [0]:
print(f'layer_block:\n{layer_block}\n')

print(f'layer_block.weights:\n{layer_block.weights}')

layer_block:
Serial[ Relu LayerNorm ]

layer_block.weights:
[(), (DeviceArray([1., 1., 1., 1., 1.], dtype=float32), DeviceArray([0., 0., 0., 0., 0.], dtype=float32))]


### Combine with `Branch`
The `Branch` combinator arranges layers into parallel computational channels:
```
def Branch(*layers):
  """Combinator that applies a list of layers in parallel to copies of inputs.

  Each layer in the input list is applied to as many inputs from the stack
  as it needs, and their outputs are successively combined on stack.

  For example, suppose one has three layers:

    - F: 1 input, 1 output
    - G: 3 inputs, 1 output
    - H: 2 inputs, 2 outputs (h1, h2)

  Then Branch(F, G, H) will take 3 inputs and give 4 outputs:

    - inputs: a, b, c
    - outputs: F(a), G(a, b, c), h1, h2    where h1, h2 = H(a, b)
```

Residual blocks, for example, are implemented using `Branch`:
```
def Residual(*layers, **kwargs):
  """Wraps a series of layers with a residual connection.

  Args:
    *layers: One or more layers, to be applied in series.
    **kwargs: If empty (the usual case), the Residual layer computes the
        element-wise sum of the stack-top input with the output of the layer
        series. If non-empty, the only key should be 'shortcut', whose value is
        a layer that applies to a copy of the inputs and (elementwise) adds its
        output to the output from the main layer series.

  Returns:
      A layer representing a residual connection paired with a layer series.
  """
  shortcut = kwargs.get('shortcut')  # default None signals no-op (copy inputs)
  layers = _ensure_flat(layers)
  layer = layers[0] if len(layers) == 1 else Serial(layers)
  return Serial(
      Branch(shortcut, layer),
      Add(),  # pylint: disable=no-value-for-parameter
  )
```

Here's a simple code example to highlight the mechanics.

### Example 5. Branch

In [0]:
relu = tl.Relu()
times_10 = tl.Fn("times_10", lambda x: x * 10.0)
branch_relu_t10 = tl.Branch(relu, times_10)

x = floats_in_range(-7, 8).reshape(3, -1)
branch_relu_t10.init(signature(x))

y1, y2 = branch_relu_t10(x)

# Show input, outputs, and two layer properties.
print(f'x:\n{x}\n\n'
      f'y1:\n{y1}\n\n'
      f'y2:\n{y2}\n\n'
      f'number of inputs expected by this layer: {branch_relu_t10.n_in}\n'
      f'number of outputs promised by this layer: {branch_relu_t10.n_out}')

x:
[[-7. -6. -5. -4. -3.]
 [-2. -1.  0.  1.  2.]
 [ 3.  4.  5.  6.  7.]]

y1:
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 2.]
 [3. 4. 5. 6. 7.]]

y2:
[[-70. -60. -50. -40. -30.]
 [-20. -10.   0.  10.  20.]
 [ 30.  40.  50.  60.  70.]]

number of inputs expected by this layer: 1
number of outputs promised by this layer: 2


# 2. Data Streams

The Trax runtime supports the concept of multiple data streams, which gives individual layers flexibility to:
  - process a single data stream ($n_{in} = n_{out} = 1$),
  - process multiple parallel data streams ($n_{in} = n_{out} = 2, 3, ... $),
  - split or inject data streams ($n_{in} < n_{out}$), or
  - merge or remove data streams ($n_{in} > n_{out}$).

We saw in section 1 the example of `Residual`, which involves both a split and a merge:
```
  ...
  return Serial(
      Branch(shortcut, layer),
      Add(),
  )
```
In other words, layer by layer:
  - `Branch(shortcut, layers)`: makes two copies of the single incoming data stream, passes one copy via the shortcut (typically a no-op), and processes the other copy via the given layers, applied in series. [$n_{in} = 1$, $n_{out} = 2$]
  - `Add()`: combines the two streams back into one by adding elementwise. [$n_{in} = 2$, $n_{out} = 1$]

# 3. Data Stack

# 4. Defining New Layer Classes

## Simpler layers, with the `Fn` layer-creating function.

## Full subclass definitions, where necessary

# 5. Models